In [18]:
import pandas as pd
import json

In [19]:
df = pd.read_csv("/data/raw_stock_ind.csv")

In [20]:
def json_parser(x):
    try:
        return json.loads(x)["symbol"]
    except Exception as e:
        print(e)
        return ""

In [21]:
df["company_symbol"] = df["info_json"].map(lambda x: json_parser(x))  

In [27]:
df["company_symbol"].unique()

array(['LTIM.NS', '20MICRONS.NS', '21STCENMGM.NS', '3MINDIA.NS',
       '3PLAND.NS', '5PAISA.NS', '63MOONS.NS', 'A2ZINFRA.NS', 'AAKASH.NS',
       'AARON.NS', 'AARTIDRUGS.NS', 'AARTIIND.NS', 'AARVEEDEN.NS',
       'AARVI.NS', 'AAVAS.NS', 'ABAN.NS', 'ABB.NS', 'POWERINDIA.NS',
       'ACC.NS', 'ACCELYA.NS', 'ACCURACY.NS', 'ACEINTEG.NS'], dtype=object)

In [9]:
df

,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBIT,Net Interest Income,...,Stock Splits,Stock_Date,info_json,Adj Close,Interest Income,Interest Income Non Operating,Write Off,Selling General And Administration,General And Administrative Expense,company_symbol
0,0.000000e+00,0.250000,1.767100e+10,NaN,NaN,1.151500e+10,1.847000e+09,5.598700e+10,1.582400e+10,-460000000.0,...,0.0,2023-06-30,"{""address1"": ""L&T House"", ""address2"": ""Ballard...",NaN,NaN,NaN,NaN,NaN,NaN,
1,0.000000e+00,0.228569,1.795100e+10,1.667000e+09,1.667000e+09,1.113700e+10,3.073000e+09,5.580700e+10,1.487800e+10,-436000000.0,...,0.0,2023-03-31,"{""address1"": ""L&T House"", ""address2"": ""Ballard...",NaN,NaN,NaN,NaN,NaN,NaN,
2,-1.182283e+08,0.236457,1.576600e+10,-5.000000e+08,-5.000000e+08,1.000500e+10,1.781000e+09,5.472200e+10,1.348500e+10,-379000000.0,...,NaN,NaN,"{""address1"": ""L&T House"", ""address2"": ""Ballard...",NaN,NaN,NaN,NaN,NaN,NaN,
3,0.000000e+00,0.245756,1.051000e+10,NaN,NaN,6.793000e+09,1.308000e+09,3.084900e+10,9.202000e+09,-189000000.0,...,0.0,2022-09-30,"{""address1"": ""L&T House"", ""address2"": ""Ballard...",NaN,NaN,NaN,NaN,NaN,NaN,
4,0.000000e+00,0.254524,9.749000e+09,NaN,NaN,6.335000e+09,1.065000e+09,2.872300e+10,8.684000e+09,-174000000.0,...,0.0,2022-06-30,"{""address1"": ""L&T House"", ""address2"": ""Ballard...",NaN,NaN,NaN,NaN,NaN,NaN,
5,0.000000e+00,0.292948,1.731410e+08,0.000000e+00,0.000000e+00,9.302400e+07,NaN,9.829390e+08,1.731410e+08,-41343000.0,...,0.0,2023-03-31,"{""address1"": ""347, GIDC Industrial Estate"", ""a...",NaN,NaN,NaN,NaN,NaN,NaN,
6,0.000000e+00,0.269854,1.452270e+08,0.000000e+00,0.000000e+00,7.380400e+07,NaN,8.279530e+08,1.452270e+08,-44128000.0,...,NaN,NaN,"{""address1"": ""347, GIDC Industrial Estate"", ""a...",NaN,NaN,NaN,NaN,NaN,NaN,
7,0.000000e+00,0.257810,2.101080e+08,0.000000e+00,0.000000e+00,1.187470e+08,NaN,9.868660e+08,2.101080e+08,-48974000.0,...,0.0,2022-09-30,"{""address1"": ""347, GIDC Industrial Estate"", ""a...",NaN,NaN,NaN,NaN,NaN,NaN,
8,0.000000e+00,0.252258,2.210800e+08,0.000000e+00,0.000000e+00,1.326160e+08,NaN,9.776720e+08,2.210800e+08,-43187000.0,...,0.0,2022-06-30,"{""address1"": ""347, GIDC Industrial Estate"", ""a...",NaN,NaN,NaN,NaN,NaN,NaN,
9,0.000000e+00,0.000000,-2.763100e+07,0.000000e+00,0.000000e+00,-2.774700e+07,1.160000e+05,3.595500e+07,-2.774700e+07,0.0,...,0.0,2023-06-30,"{""address1"": ""Grease House"", ""address2"": ""1st ...",NaN,NaN,NaN,NaN,NaN,NaN,
